# Proposal

In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import altair as alt
from ucimlrepo import fetch_ucirepo 
  
# Fetching the dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets 

ConnectionError: Error connecting to server

## Introduction
Heart disease has often been attributed to two broad reasons in the field of medical sciences; lifestyle and genetics. Whilst it is difficult to obtain data, extrapolate and correctly predict the possibility that an individual develops a heart disease owing to genetics, it is easier to do so by looking at their lifestyle and present health. 

This project aims at predicting the the prevalence of heart disease in patients based on a number of lifestyle variables that provide us insight into the individual's current health. The data set classifies the presence of heart disease based on 5 categories, ranking from 0 (absence) to 4 (highest prevelance), and we aim to correctly predict the class of a new observation based on the variables we have.It provides us with 13 explanatory variables and 303 observations that we would use as inputs to a KNN model. 

Our aim is therefore not only to create a relevant KNN model, but also to train and test it and improve it's accuracy such that the probability of a new observation to be rightly predicted is sufficiently high. 

## Preliminary exploratory data analysis

In [ ]:
heart_disease.variables

Python data containing organized information is shown below. The data is in tidy format, with columns renamed.

In [ ]:
X = X.rename(columns = {
    "trestps" : "resting_blood_pressure", 
    "chol" : "serum_cholestoral", 
    "fbs" : "fasting_blood_sugar_greater_than_120_mg/dl", 
    "thalach" : "maximum_heart_rate_achieved", 
    "exang" : "exercise_induced_angina", 
    "oldpeak" : "ST_depression_induced_by_exercise_relative_to_rest", 
    "ca" : "number_of_major_vessels"
}).drop(columns = ["cp", "restecg", "slope", "thal"])

X

In [ ]:
y

Table showing the data types of each class:

In [ ]:
X.info()

Table showing the number of observations (count), mean, standard deviation (std), and other statistics: 

In [ ]:
X.describe()

Histogram of distribution of age:

In [ ]:
# combine X and y
Xy = X.assign(presence_of_heart_disease = y)

age_distribution = alt.Chart(Xy, title = "Distribution of age and severity of heart disease").mark_bar().encode(
    x = alt.X("age").title("Age (in years)"), 
    y = alt.Y("count()").title("Counts of Age"), 
    color = alt.Color("presence_of_heart_disease:N").title([
      "Presence of Heart Disease",
      "(with 0 being absent and",
      "4 being most severe)"
    ])
)

age_distribution

In [ ]:
# distribution of gender and age for people with heart disease

# filter for presence_of_heart_disease != 0
Xy_with_disease = Xy[Xy["presence_of_heart_disease"] != 0]

age_gender_dist = alt.Chart(Xy_with_disease, title = "Age and gender of people with heart disease").mark_bar().encode(
    x = alt.X("age").title("Age (in years)"), 
    y = alt.Y("count()").title("Counts of Age (of people with heart disease)"), 
    color = alt.Color("sex:N").title("Gender")
)

age_gender_dist

## Methods

In [3]:
X.describe()

NameError: name 'X' is not defined

We can see that the dataset is already pretty clean. Indeed, we can see that the sex is already encoded as a binary feature such as all the other features.

We couldn't identify in the dataset useless features (such as "id") that do not give any information for the prediction. However, some features were not documented enough or at all so we decided to drop them such as:
- `cp`
- `restecg`
- `slope` 
- `thal`

As we can see on the descibe table, the features of the dataset are not scaled yet We will need to scale the features in order to be able to perform kNN classification correctly. To do so we are going to use `StandardScaler` as seen in class.

We are planning on using `GridSearchCV` in order to perform multiple cross-valdiation and find a great value for the hyperparameter `n_neighbors`.

## Expected outcomes and significance

This project aims to predict the possibility of an individual contracting heart disease due to their lifestyle and what behaviours are more likely to lead to heart disease. This analysis of the result should lead to a system that can predict heart disease by factors such as age, diet, or sex. Given the preliminary results, heart disease is more likely to be present in older individuals and those with higher blood sugar. As the project moves forward, the goal is to refine those results into the class system referenced in the introduction. If successful, this model could help health care professionals predict which patients need specific care in order to prevent heart disease and could even be a self-assessment tool for individuals that may want to check their propensity for the disease. Some questions that may come from the project could be looking at medical dataset and their biases. As with all data, this information will reflect the biases all around us. In situations where an informatic model can actively hurt individuals, a larger dataset could be useful to balance out this phenomenon. It may also lead to questions of what treatment plan may be effective considering the individual factors used to assess the person’s likelihood to contract the disease. A person’s health varies greatly between individuals, but also between the spheres variables being used in this project. These questions could further the usefulness of this project. 